# Rekurentní neuronové sítě

Většina metod strojového učení předpokládá, že jednotlivé instance jsou nezávislé. Tento předpoklad je velmi často validní a pomáhá m odely zjednodušit, ale někdy se může stát, že potřebuje pracovat s daty, ve kterých nějaké další závislosti jsou, např. v ligvistice pracujeme s texty, které jsou uspořádané poslouplnosti slov a na jejich pořadí záleží, nebo můžeme pracovat s videem, kde jednotlivé obrázky za sebou také nejsou nezávislé. Někdy jde tyto závislosti ignorovat (např. pokud ve videu máme v každém obrázku něco najít), ale obecně to tak být nemusí a klíč ke správné klasifikaci může ležet právě v posloupnosti vstupů.

Rekuretní neuronové sítě pracují explicitně s upořádáním dat. Kromě dopředných hran obsahují navíc i zpětné hrany, které předávají informaci do dalšího časového okamžiku (typicky do doby předložení dalšího vzoru).

Trénování rekturentních sítí se moc neliší od trénování feed-forwards sítí, jen je potřeba síť tzv. rozbalit v čase, tj. nakopírovat ji přes několik časových kroků s tím, že rekurentní hrany vedou mezi těmito kopiemi a všechny váhy sjou sdílené. Rozvinutá síť neobsahuje žádné cykly a je možné ji trénovat pomocí back-propagation. Tato verze back-propagation se také nazývá back-propagation through time (BPTT).

Problém při trénování rekuretních sítí způsobuje to, že se gradienty musí propagovat přes mnoho vrstev v rozvinuté síti (proto se také rekurentní sítě dají považovat za hluboké sítě). To vede k tomu, že gradienty buď konvergují k nule, nebo naopak "explodují". To, která varianta nastane závisí na konkrétních hodnotách gradientů, ale např. v síti s jedinou rekurentní hranou dojde ke konvergenci k nule, když je gradient menší než jedna a k explozi, když je větší než jedna. Tedy na jeden z problémů narazíme téměř vždy.

## Long-Short Term Memory (LSTM)

LSTM sítě řeší problém mizijícího nebo explodujícího gradientu tím, že nahradí každý neuron novou jednotkou. V ní je rekurentní váha, která je vždy nastavena na jedna. Obrázek jedné jednotky (memory cell) z LSTM sítí je níže (převzato z [1]).

![memory cell](memory_cell.png)

V každé buňce jsou kromě vstupu ještě tři brány -- vstupní, výstupní a zapomínací, ty mají za úkol pracovat s vnitřní pamětí buňky a ovlivňují, co se do ní uloží, případně co se z ní smaže. Zároveň ovlivňují i hodnotu výstupu. Každá z bran se chová jako jeden běžný neuron, tj. spočítá vážený součet vstupů a aplikuje na něj nejlineární funkci. Vnitřní stav buňky se chová jako neuron s dvěma vstupy a lineární aktivací, v zásadě tedy počítá vážený součet svojí předchozí aktivaci a svojí současné aktivace (váhy jsou dané vstupní a zapomínací branou). Výstup celé buňky je potom hodnota vnitřního stavu pronásobená hodnotou výstupní brány, případně se na něj ještě může aplikovat nějaké nelineární funkce.

In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.datasets.data_utils import get_file
import numpy as np
import random
import sys

'''
    Example script to generate text from Nietzsche's writings.
    At least 20 epochs are required before the generated text
    starts sounding coherent.
    It is recommended to run this script on GPU, as recurrent
    networks are quite computationally intensive.
    If you try this script on new data, make sure your corpus
    has at least ~100k characters. ~1M is better.
'''

path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print('corpus length:', len(text))

chars = set(text)
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for iteration in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

corpus length: 600893
total chars: 57
nb sequences: 200291
Vectorization...
Build model...

--------------------------------------------------
Iteration 1
Epoch 1/1
200291/200291 [==============================] - 332s - loss: 2.8894   

----- diversity: 0.2
----- Generating with seed: "imself
because weari"
imself
because wearis the the the the the ses and ante the ane the the the the and the the the the the the the se the the the the the the the the the the the the the se the the the the the the and ante the the the se the the the the the the the the the an an in the the the the the the the the the the the the the the the the the the the the the ane the an the the se the the the the the the the the the the he the the t

----- diversity: 0.5
----- Generating with seed: "imself
because weari"
imself
because wearig on ge the an the monl ole tas th ant the peathe toe the sos ant thor this te an the the an seth ins an innind in s ans oritin boslechecind ansthe th anging whe tho ther te sh

Using gpu device 0: GRID K520


KeyboardInterrupt: 

## Bi-directional Recurent Neural Networks (BRNN)

V BRNN jsou dvě skryté vrstvy, obě napojené přímo na vstupy a výstupy. Neurony z jedné skryté vrstvy propagují informaci dopředu v čase (jako u běžné neuronové sítě). Neurony v druhé skryté vrstvě propagují informaci zpět v čase (tj. později předložené vzory mohou ovlivnit výstup u dříve předložených vzorů). Celá síť se zase po rozvinutí v čase dá trénovat pomocí BPTT. 

Právě propagace informace zpět v čase je na této síti zajímavá. Obecně nemáme k dispozici budoucí pozorování a nemůžeme je využít, ale existují situace (například v lingvistice při určování slovních druhů ve větách), kdy máme současné i budoucí informace najednou (v tomto případě za budoucí informaci považujeme další slova ve větě, větu typicky máme celou). V takových případech je vyžití informace z budoucích stavů prospěšné.

## Neural Turing Machines (NTM)

NTM rozšiřují myšlenku ukládání stavu z LSTM a přidávají adresovatelnou paměť. Stejně jako Turingovy stroje mají čtecí a zapisovací hlavu. Zajímavé na nich je, že celá práce s hlavami a s pamětí je navržena tak, že ji lze zderivovat a síť učit pomocí back-propagation. Koho by to více zajímalo, nechť se podívá na článek [2].

## Reference

[1] Lipton, Zachary C. "A Critical Review of Recurrent Neural Networks for Sequence Learning." arXiv preprint arXiv:1506.00019 (2015).

[2] Graves, Alex, Greg Wayne, and Ivo Danihelka. "Neural Turing Machines." arXiv preprint arXiv:1410.5401 (2014).